In [1]:

from flask import Flask,Response
from flask import request,jsonify
from flask_sqlalchemy import SQLAlchemy
from flask import current_app
import pandas as pd
import subprocess
import json
import os
import sys
current_directory = os.getcwd()
base_directory = os.path.join(current_directory, '../../common/backend/')
base_directory = os.path.normpath(base_directory)  
sys.path.insert(1, base_directory)
from model import * 
from util import *
# %matplotlib widget
import seaborn as sns
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time 
from sqlalchemy.orm import aliased
from sqlalchemy import asc
from functools import lru_cache
import json
import numpy as np
import pymysql
import configparser
from sqlalchemy import func, distinct

config_path = os.path.join(current_directory, "../../config/qaas-web.conf")
config = configparser.ConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read(config_path)
lore_database_uri = config['web']['SQLALCHEMY_DATABASE_URI_LORE']
db = SQLAlchemy()
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = lore_database_uri
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db.init_app(app)
def with_app_context(func):
    def wrapper(*args, **kwargs):
        with app.app_context():
            return func(*args, **kwargs)
    return wrapper

In [2]:
# @with_app_context
# def lore_metrics_table_test():
#     src_loop_id = 450656 
#     vendor = "icc" 
#     version = "17.0.1"
#     src_loop = db.session.query(SrcLoop).filter_by(table_id = src_loop_id).one()
#     metrics = src_loop.loops[0].lore_loop_measures[0].lore_loop_measure_metrics
#     for metric in metrics:
#         key = metric.metric_name
#         value = metric.metric_value
#         # print(key)
#         try:
#             float_value = float(value)
#             # print(key, value)
#             print(key)

#         except:            
#             # print(key, value)
#             pass
# lore_metrics_table_test()

In [3]:
# @with_app_context
# def souce_metrics_table_test():
#     src_loop_id = 450656 
#     vendor = "icc" 
#     version = "17.0.1"
#     src_loop = db.session.query(SrcLoop).filter_by(table_id = src_loop_id).one()
#     metrics = src_loop.source.source_metrics
#     print(type(metrics))
#     for metric in metrics:
#         key = metric.metric_name
#         value = metric.metric_value
#         # print(key)
#         try:
#             float_value = float(value)
#             # print(key, value)
#             print(key)

#         except:            
#             # print(key, value)
#             pass
# souce_metrics_table_test()

In [4]:
#data cleanning
file_path = os.path.join(current_directory, 'compiler_prediction_model_data.csv')
if os.path.exists(file_path):
    df = pd.read_csv(file_path, sep=',')



In [5]:
df[['original_loop_id','base_median', 'rating','is_best_compiler']].head(20)

,original_loop_id,base_median,rating,is_best_compiler
0,1,768.0,0.050000,0.0
1,1,736.0,0.062500,0.0
2,1,724.0,0.083333,0.0
3,1,752.0,0.058824,0.0
4,1,728.0,0.076923,0.0
5,1,812.0,0.045455,0.0
6,1,896.0,0.043478,0.0
7,1,928.0,0.041667,0.0
8,1,728.0,0.076923,0.0
9,1,728.0,0.076923,0.0


In [6]:
df.head()

,original_loop_id,mutated_loop_id,mutation_number,compiler_vendor_version,restructured,compiled,exe_passed,tiny,pluto,n_mutation_attempts,...,o3_pin_vector_st_cnt,o3_pin_fp_arith_inst_cnt,o3_pin_fp_sse_inst_cnt,o3_pin_fp_avx_inst_cnt,o3_pin_int_sse_inst_cnt,o3_pin_int_avx_inst_cnt,o3_pin_vector_inst_exclude_mem_cnt,o3_pin_vector_other_inst_cnt,is_best_compiler,rating
0,1,9,1,clang_3.4.2,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000
1,1,10,1,clang_3.6.2,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062500
2,1,11,1,clang_4.0.0,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333
3,1,12,1,gcc_4.7.4,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058824
4,1,13,1,gcc_4.8.5,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923


In [7]:
for col in df.columns:
    print(col)

original_loop_id
mutated_loop_id
mutation_number
compiler_vendor_version
restructured
compiled
exe_passed
tiny
pluto
n_mutation_attempts
nest_level
stmt_cnt
mem_access_cnt
fp_bin_op_cnt
int_bin_op_cnt
int_un_op_cnt
assign_cnt
const_cnt
const_0_cnt
const_1_cnt
bin_op_with_int_const_cnt
bb_cnt
bb_1_succ_cnt
bb_2_succ_cnt
bb_3_plus_succ_cnt
bb_1_pred_cnt
bb_2_pred_cnt
bb_3_plus_pred_cnt
bb_1_pred_1_succ_cnt
bb_1_pred_2_succ_cnt
bb_2_pred_1_succ_cnt
bb_2_pred_2_succ_cnt
bb_3_plus_pred_3_plus_succ_cnt
bb_5_minus_stmt_cnt
bb_6_to_10_stmt_cnt
bb_11_plus_stmt_cnt
cfg_edge_cnt
has_branch
count_ins
profiling
base_sanity
novec_sanity
sse_sanity
avx_sanity
avx2_sanity
o3_sanity
base_min
base_max
base_mean
base_median
base_sd
novec_min
novec_max
novec_mean
novec_median
novec_sd
sse_min
sse_max
sse_mean
sse_median
sse_sd
avx_min
avx_max
avx_mean
avx_median
avx_sd
avx2_min
avx2_max
avx2_mean
avx2_median
avx2_sd
o3_min
o3_max
o3_mean
o3_median
o3_sd
interchange_order
tiling_order
shared
for_comparison

In [8]:
columns_to_drop = ['o3_min', 'o3_max', 'o3_mean', 'o3_median', 'avx2_median', 'o3_median', 'avx2_sd', 'restructured', 'compiled',
'exe_passed','tiny','pluto','base_sanity','novec_sanity', 'sse_sanity', 'avx_sanity', 'avx2_sanity', 'o3_sanity', 'base_min','base_max','base_mean','base_sd','novec_min','novec_max',
'novec_mean','novec_median','novec_sd','sse_min','sse_max','sse_mean','sse_median','sse_sd','avx_min','avx_max','avx_mean','avx_median','avx_sd','avx2_min','avx2_max','avx2_mean','avx2_median',
'avx2_sd','o3_min','o3_max','o3_mean','o3_median','o3_sd','interchange_order','tiling_order','shared','for_comparison']

In [9]:
df = df.drop(columns_to_drop, axis=1)


In [10]:
#when is gcc a winner
gcc_best_compilers = df[(df['compiler_vendor_version'].str.contains("gcc")) & (df['is_best_compiler'] == 1)]
gcc_best_compilers

,original_loop_id,mutated_loop_id,mutation_number,compiler_vendor_version,n_mutation_attempts,nest_level,stmt_cnt,mem_access_cnt,fp_bin_op_cnt,int_bin_op_cnt,...,o3_pin_vector_st_cnt,o3_pin_fp_arith_inst_cnt,o3_pin_fp_sse_inst_cnt,o3_pin_fp_avx_inst_cnt,o3_pin_int_sse_inst_cnt,o3_pin_int_avx_inst_cnt,o3_pin_vector_inst_exclude_mem_cnt,o3_pin_vector_other_inst_cnt,is_best_compiler,rating
2212,2237,2264,3,gcc_4.7.4,4.0,2.0,8.0,3.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2260,2301,2328,3,gcc_4.7.4,16.0,1.0,4.0,2.0,1.0,7.0,...,437.0,443.0,0.0,439.0,0.0,1.0,440.0,1311.0,1.0,1.0
2366,2429,2440,1,gcc_6.2.0,4.0,1.0,7.0,4.0,14.0,7.0,...,32766.0,131070.0,0.0,32767.0,0.0,0.0,32767.0,98298.0,1.0,1.0
2400,2453,2482,3,gcc_6.2.0,4.0,2.0,9.0,2.0,0.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2430,2485,2520,4,gcc_4.7.4,32.0,1.0,5.0,2.0,0.0,7.0,...,478.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553378,565332,565336,1,gcc_6.2.0,16.0,1.0,4.0,2.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
553427,565380,565393,1,gcc_6.2.0,16.0,1.0,5.0,5.0,10.0,4.0,...,0.0,256368.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
553595,565564,565585,2,gcc_6.2.0,4.0,2.0,9.0,7.0,0.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
553781,565772,565799,3,gcc_4.7.4,4.0,2.0,8.0,3.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
#predict rating for each loop then see which compiler it predicted has highest rating to know what is the best compielr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
import numpy as np
from sklearn.preprocessing import LabelEncoder

data = df.dropna().head(1000)
#reduce memory
# categorical_cols = ['compiler_vendor_version']

# label_encoder = LabelEncoder()
# data['compiler_vendor_version_encoded'] = label_encoder.fit_transform(data[categorical_cols])
# data = data.drop(categorical_cols, axis=1)
# numeric_cols = [col for col in data.columns if col not in ['base_median', 'is_best_compiler', 'rating', 'novec_median', 'sse_median', 'avx_median', 'avx2_median', 'o3_median', 'compiler_vendor_version']]
# len(numeric_cols)
# numeric_data = data[numeric_cols]
# X = numeric_data
# y = data['rating']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# encode categorical variables
categorical_cols = ['compiler_vendor_version']
encoder = OneHotEncoder(sparse=False)
encoded_categorical_columns = encoder.fit_transform(data[categorical_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical_columns, columns=encoder.get_feature_names_out(categorical_cols))

numeric_data = data.drop(categorical_cols + ['base_median', 'is_best_compiler','rating'], axis=1)
processed_data = pd.concat([numeric_data, encoded_categorical_df], axis=1)

X = processed_data
y = data['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#random forest regression
from sklearn.ensemble import RandomForestRegressor 
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the regressor on the training set
regressor.fit(X_train, y_train)

# Predict ratings on the test set
y_pred = regressor.predict(X_test)

# Evaluate the regressor
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')
# feature importance
feature_importances = regressor.feature_importances_
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
print(importance_df.sort_values(by='Importance', ascending=False))

In [ ]:
#see what is the predictied best compielr
X_test_with_predictions = X_test.copy()
X_test_with_predictions['predicted_rating'] = y_pred

X_test_with_predictions['predict_best_compiler'] = 0
for _, group in X_test_with_predictions.groupby('original_loop_id'):
    # row indwx with the min base_median
    best_compiler_idx = group['predicted_rating'].idxmax()
    X_test_with_predictions.at[best_compiler_idx, 'predict_best_compiler'] = 1

X_test_with_predictions[['original_loop_id', 'predicted_rating', 'compiler_vendor_version_clang_3.6.2',
       'compiler_vendor_version_clang_4.0.0',
       'compiler_vendor_version_gcc_4.7.4',
       'compiler_vendor_version_gcc_4.8.5',
       'compiler_vendor_version_gcc_6.2.0',
       'compiler_vendor_version_icc_15.0.6',
       'compiler_vendor_version_icc_17.0.1','predict_best_compiler']]

In [ ]:
X_test_with_predictions[['original_loop_id','mutation_number', 'predicted_rating','predict_best_compiler']]

In [ ]:
# random forest
# from sklearn.metrics import accuracy_score, classification_report

# model = RandomForestClassifier(n_estimators=100, random_state=42)

# # train
# model.fit(X_train, y_train)
# # predictions
# predictions = model.predict(X_test)
# print(predictions)
# print(y)
# # lose function
# accuracy = accuracy_score(y_test, predictions)
# report = classification_report(y_test, predictions)
# print(f'Accuracy: {accuracy}')
# print(report)
# feature importance
# feature_importances = model.feature_importances_
# importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
# print(importance_df.sort_values(by='Importance', ascending=False))


In [ ]:
#check what data we hava in lore dynamic metrics table
@with_app_context
def value_test():
    src_loop_id = 1
 
    vendor = "icc" 
    version = "17.0.1"
    src_loop = db.session.query(SrcLoop).filter_by(table_id = src_loop_id).one()
    compiler = src_loop.loops[0].compiler_option.compiler
    print(src_loop.mutation_number, src_loop.execution.application.workload, src_loop.execution.application.version, src_loop.execution.application.program, compiler.vendor, compiler.version, src_loop.file)
value_test()